! rm -rf metastore_db/

spark.stop()

spark = SparkSession \
    .builder \
    .appName("Prediction of Loan Payment") \
    .master("spark://192.168.10.1:7077") \
    .config("spark.executor.cores","1")\
    .getOrCreate();

In [ ]:
sc

In [ ]:
! wget https://storage.googleapis.com/aekanunlab/LoanStats_web.csv

In [ ]:
! hdfs dfs -mkdir -p /rawzone/

In [ ]:
! hdfs dfs -put LoanStats_web.csv /rawzone/

# 1. Data Understanding

## Collecting the current data from the LendingClub's Website.

In [1]:
#! rm -rf LoanStats_web.csv

In [2]:
#! touch LoanStats_web.csv

In [3]:
! ls -l ./LoanStats_web.csv

-rw-r--r-- 1 root root 1160157683 Sep  4  2019 ./LoanStats_web.csv


In [4]:
! head -3 ./LoanStats_web.csv

"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","open_il_12m","open_il_24m","mths_since_rcnt_il","total_bal_il","il_util","open_rv_1

In [5]:
! wc -l ./LoanStats_web.csv

1432493 ./LoanStats_web.csv


In [6]:
raw_df = spark.read.format('csv').option('header','true').option('mode','DROPMALFORMED')\
.load('/rawzone/LoanStats_web.csv')

In [7]:
raw201617_df = raw_df

In [8]:
raw201617_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

### Numbers of Fields [1]

In [9]:
len(raw201617_df.columns)

144

In [10]:
rawweb_df = raw201617_df.select('id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 #'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m')

### Numbers of Fields after attributes selections [1]

In [11]:
len(rawweb_df.columns)

73

In [12]:
rawweb_df.select(['issue_d']).distinct().show()

+--------+
| issue_d|
+--------+
|Oct-2016|
|Feb-2019|
|Mar-2018|
|Sep-2018|
|    null|
|Jan-2016|
|Feb-2017|
|Nov-2018|
|Jul-2016|
|Aug-2016|
|May-2016|
|Jan-2018|
|Jul-2018|
|Apr-2018|
|Dec-2018|
|Feb-2018|
|Aug-2018|
|Oct-2018|
|Apr-2016|
|Sep-2016|
+--------+
only showing top 20 rows



In [13]:
raw_df = rawweb_df

In [14]:
raw_df.count()

1432466

In [15]:
df_colfam5 = raw_df.select("loan_amnt","term","int_rate","installment","grade","emp_length",\
                           "home_ownership","annual_inc","verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs","earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d")

### Numbers of Fields after final attributes selections [1]

In [16]:
len(df_colfam5.columns)

22

In [17]:
! pip install pandas

In [18]:
import pandas as pd
pd.DataFrame(df_colfam5.describe().take(6), columns=df_colfam5.describe().columns).transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
loan_amnt,1432440,15370.388358325654,9646.026272413836,1000,9975
term,1432440,None,None,36 months,60 months
int_rate,1432440,None,None,5.31%,30.99%
installment,1432440,456.68909888718275,281.7139997355062,100.01,999.98
grade,1432440,None,None,A,G
emp_length,1432440,None,None,1 year,n/a
home_ownership,1432440,None,None,ANY,RENT
annual_inc,1432440,81034.58293296752,134183.3569671438,0,99999.84
verification_status,1432440,None,None,Not Verified,Verified


df_colfam5.select(['last_credit_pull_d']).distinct().\
orderBy(['last_credit_pull_d'],ascending=1).show()

df_colfam5.select(['last_credit_pull_d']).\
filter(df_colfam5.last_credit_pull_d == '').count()

df_colfam5.select(['last_credit_pull_d']).\
filter(df_colfam5['last_credit_pull_d'].isNull()).count()

In [19]:
df_colfam5.groupby('home_ownership').count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|          null|    26|
|           OWN|170888|
|          RENT|556961|
|      MORTGAGE|701646|
|           ANY|  2940|
|          NONE|     5|
+--------------+------+



# 2. Data Preparation

Divide this process into 2 parts. The first is a business oriented preparation that turn many business rules to be programming's logics. Its result benefits many tasks related to the Business Intelligence and other descriptive analytics. The second one is a data science oriented preparation that turn many requirements of data science to be programming's logics. Its result benefits many tasks related to the Predictive analytics.

Asumption: Business oriented preparation: No missing values, ONLY month, Correct data types.

## Data Cleansing: Remove missing values

In [20]:
df_no_missing = df_colfam5.dropna(how='any')

## Check for a schema

In [21]:
df_no_missing.printSchema()

root
 |-- loan_amnt: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [22]:
df_no_missing_fitmem = df_no_missing.repartition(60)

In [23]:
df_no_missing_cached = df_no_missing_fitmem.cache()

In [24]:
df_no_missing_cached.registerTempTable("df")

21/11/25 11:06:08 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Number of data rows

In [25]:
df_no_missing_cached.count()

1429645

## Data Transformation: Remove Sign of Percent and Extract Month.

In [26]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

### Remove Sign of Percent

In [27]:
def f_removepercentsign(origin):
    return origin.rstrip('%')

In [28]:
removepercentsign = udf(lambda x: f_removepercentsign(x),StringType())

### Extract Month

In [29]:
def f_extractmonth(origin):
    return origin.split('-')[0]

In [30]:
extractmonth = udf(lambda x: f_extractmonth(x),StringType())

In [31]:
extractterm = udf(lambda x: x.replace('months',''),StringType())

In [32]:
#extractemp_length = udf(lambda x: x.replace('year',''),StringType())

In [33]:
from pyspark.sql.functions import col

In [34]:
dfWithCrunch = df_no_missing_cached.\
withColumn('revol_util',removepercentsign(col('revol_util')).cast(DoubleType())).\
withColumn('int_rate',removepercentsign(col('int_rate')).cast(DoubleType())).\
withColumn('earliest_cr_line',extractmonth(col('earliest_cr_line')).cast(StringType())).\
withColumn('last_credit_pull_d',extractmonth(col('last_credit_pull_d')).cast(StringType())).\
withColumn('dti',col('dti').cast(DoubleType())).\
withColumn('loan_amnt',col('loan_amnt').cast(DoubleType())).\
withColumn('revol_bal',col('revol_bal').cast(DoubleType())).\
withColumn('term',extractterm(col('term')).cast(DoubleType())).\
withColumn('installment',col('installment').cast(DoubleType())).\
withColumn('open_acc',col('open_acc').cast(DoubleType())).\
withColumn('total_acc',col('total_acc').cast(DoubleType())).\
withColumn('pub_rec',col('pub_rec').cast(DoubleType())).\
withColumn('annual_inc',col('annual_inc').cast(DoubleType()))

In [35]:
#dfWithCrunch.select(['last_credit_pull_d']).printSchema()

In [36]:
#dfWithCrunch.groupby('emp_length').count().show()

In [37]:
#dfWithCrunch.count()

In [38]:
dfWithCrunch.select(['loan_amnt','int_rate','dti','home_ownership','grade','term']).show()

+---------+--------+-----+--------------+-----+----+
|loan_amnt|int_rate|  dti|home_ownership|grade|term|
+---------+--------+-----+--------------+-----+----+
|   6400.0|   16.99|14.34|      MORTGAGE|    D|36.0|
|   8500.0|   10.49|12.08|          RENT|    B|36.0|
|   6000.0|    8.24|17.39|          RENT|    B|36.0|
|   8000.0|    7.99|24.06|          RENT|    A|36.0|
|   5000.0|   11.44|11.46|      MORTGAGE|    B|36.0|
|  16000.0|   30.49|22.21|          RENT|    F|60.0|
|  28000.0|   15.99|10.97|      MORTGAGE|    C|60.0|
|  12000.0|   10.99|21.42|      MORTGAGE|    B|60.0|
|   4000.0|   11.49| 5.88|      MORTGAGE|    B|36.0|
|  20000.0|   14.99| 8.71|      MORTGAGE|    C|36.0|
|  17000.0|   10.49|20.96|          RENT|    B|36.0|
|  16000.0|   12.74| 17.8|      MORTGAGE|    C|36.0|
|  10000.0|    6.99| 4.73|      MORTGAGE|    A|36.0|
|   2000.0|    6.99|12.71|      MORTGAGE|    A|36.0|
|   9000.0|   14.99|20.88|      MORTGAGE|    C|36.0|
|  30000.0|   16.99| 31.1|          RENT|    D

In [39]:
#dfWithCrunch.select(['loan_amnt','int_rate','dti','home_ownership','grade']).printSchema()

In [40]:
dfWithCrunch.select(['loan_amnt','annual_inc']).describe().show()

+-------+------------------+------------------+
|summary|         loan_amnt|        annual_inc|
+-------+------------------+------------------+
|  count|           1429645|           1429645|
|   mean|15366.699302973815| 81111.66344812172|
| stddev| 9641.851365996215|134222.54004920385|
|    min|            1000.0|               0.0|
|    max|           40000.0|             6.1E7|
+-------+------------------+------------------+



In [41]:
rawhive_df = dfWithCrunch.repartition(60).cache()
#rawhive_df.registerTempTable("crunched_data")

In [42]:
len(rawhive_df.columns)

22

In [43]:
rawhive_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [44]:
#rawhive_df.select(['loan_amnt','int_rate','dti','home_ownership','grade']).printSchema()

In [45]:
rawhive_df.count()

1429645

### Data Transformation: Normalization of "annual_inc" and "loan_amnt"

In [46]:
from pyspark.sql.functions import *

max_annual_inc = rawhive_df.select(max('annual_inc')).collect()[0][0]

min_annual_inc = rawhive_df.select(min('annual_inc')).collect()[0][0]

#sqlContext.udf.register("t_annual_inc", lambda x: ((x-min_annual_inc)/(max_annual_inc-min_annual_inc)))

In [47]:
def t_annual_inc(origin):
    return ((origin-min_annual_inc)/(max_annual_inc-min_annual_inc))

In [48]:
normalized_annual_inc = udf(lambda x: t_annual_inc(x),DoubleType())

In [49]:
max_loan_amnt = rawhive_df.select(max('loan_amnt')).collect()[0][0]

min_loan_amnt = rawhive_df.select(min('loan_amnt')).collect()[0][0]

#sqlContext.udf.register("t_loan_amnt", lambda x: ((x-min_loan_amnt)/(max_loan_amnt-min_loan_amnt)))

In [50]:
def t_loan_amnt(origin):
    return ((origin-min_loan_amnt)/(max_loan_amnt-min_loan_amnt))

In [51]:
normalized_loan_amnt = udf(lambda x: t_loan_amnt(x),DoubleType())

In [52]:
normalized_df = rawhive_df.withColumn('loan_amnt',normalized_loan_amnt(col('loan_amnt'))).\
withColumn('annual_inc',normalized_annual_inc(col('annual_inc')))

In [53]:
normalized_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



### Number of data rows that are only "Fully Paid" and "Charged Off"

In [54]:
from pyspark.sql.functions import col

In [55]:
normalized_df.select(['loan_amnt','term','int_rate','installment','grade','annual_inc','revol_bal','loan_status']).\
orderBy(['loan_amnt','term','int_rate','installment','grade','annual_inc']).show()

+---------+----+--------+-----------+-----+--------------------+---------+-----------+
|loan_amnt|term|int_rate|installment|grade|          annual_inc|revol_bal|loan_status|
+---------+----+--------+-----------+-----+--------------------+---------+-----------+
|      0.0|36.0|    5.31|      30.12|    A|4.098360655737705E-4|   3215.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|5.081967213114754E-4|  25993.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|5.901639344262295E-4|   3311.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|6.065573770491804E-4|   1605.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|6.557377049180328E-4|   1329.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|6.557377049180328E-4|   1016.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|7.180327868852459E-4|    804.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|9.344262295081968E-4|    858.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|

In [56]:
normalized_filtered_df = normalized_df.filter(col('loan_status') == 'Fully Paid').\
union(normalized_df.filter(col('loan_status') == 'Charged Off'))

In [57]:
data = normalized_filtered_df.repartition(60).cache()

### Drop Null

In [58]:
data_no_missing_df = data.dropna(how='any')

In [59]:
data_no_missing_df.count()

698373

# 3. Data Modeling

In [60]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder,\
VectorIndexer, QuantileDiscretizer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, NaiveBayes\
, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

In [61]:
data_no_missing_df_fully_paid = data_no_missing_df.filter(col('loan_status') == 'Fully Paid').sample(True, 0.3, 42)

In [62]:
data_no_missing_df_charge_off = data_no_missing_df.filter(col('loan_status') == 'Charged Off')

In [63]:
final_data_no_missing_df = data_no_missing_df_fully_paid.union(data_no_missing_df_charge_off)

In [64]:
training , test = final_data_no_missing_df.filter(col('loan_amnt') > 0).randomSplit([0.8,0.2])

## Calculation for Confusion Matrix.

In [65]:
def eval_metrics(lap):

    tp = float(len(lap[(lap['loan_status']=='Charged Off') &\
                       (lap['prediction_label']=='Charged Off')]))
    
    fn = float(len(lap[(lap['loan_status']=='Charged Off') &\
                       (lap['prediction_label']=='Fully Paid')]))

    tn = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                       (lap['prediction_label']=='Fully Paid')]))

    fp = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                       (lap['prediction_label']=='Charged Off')]))


    try:
        positivepredictivevalue = tp / (tp+fp)
        negativepredictivevalue = tn / (tn+fn)
        sensitivity = tp / (tp+fn)
        specificity = tn / (tn+fp)
        accuracy = (tp+tn) / (tp+tn+fp+fn)
        return {'PPV': positivepredictivevalue, 'Sensitivity': sensitivity,\
            'NPV':negativepredictivevalue, 'Specificity': specificity, 'Accuracy': accuracy}
    except ZeroDivisionError:
        return 0

    

In [66]:
! pip install pandas

# Jan2020

In [67]:
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import Bucketizer

In [102]:
def get_model(df,categoricalCols,continuousCols,\
              discretedCols,split_range,labelCol):

    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
    from pyspark.sql.functions import col
    
    
    labelIndexer = StringIndexer(inputCol=labelCol,\
                             outputCol='indexedLabel',\
                             handleInvalid='keep')

    indexers = [ StringIndexer(handleInvalid='keep',\
                               inputCol=c, outputCol="{0}_indexed".format(c))
                 for c in categoricalCols ]

    # default setting: dropLast=True
    encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(),
                 outputCol="{0}_encoded".format(indexer.getOutputCol()))
                 for indexer in indexers ]
    discretizers = [ Bucketizer(inputCol=d, outputCol="{0}_discretized".format(d)\
                 ,splits=split_range)
                 for d in discretedCols ]
    
    
    featureCols = ['features']
    assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders]
                                + continuousCols +\
                                [discretizer.getOutputCol() for discretizer in discretizers], \
                                outputCol='features')
    

    pipeline_feature = Pipeline(stages=[labelIndexer] + indexers + encoders + discretizers + \
                        [assembler])
    
    
    
    train_df_features = pipeline_feature.fit(training).transform(training)
    
    layers = [
    train_df_features.schema["features"].metadata["ml_attr"]["num_attrs"],
    20, 10, 5, 3]
    
    
    clf = MultilayerPerceptronClassifier(labelCol='indexedLabel',featuresCol='features'\
                                     ,layers=layers)
    
    
    pipeline = Pipeline(stages=[labelIndexer] + indexers + encoders + discretizers + \
                        [assembler] + [clf])
    


    model=pipeline.fit(df)


    return model

In [69]:
training

DataFrame[loan_amnt: double, term: double, int_rate: double, installment: double, grade: string, emp_length: string, home_ownership: string, annual_inc: double, verification_status: string, loan_status: string, purpose: string, addr_state: string, dti: double, delinq_2yrs: string, earliest_cr_line: string, inq_last_6mths: string, open_acc: double, pub_rec: double, revol_bal: double, revol_util: double, total_acc: double, last_credit_pull_d: string]

In [70]:
training.select('grade').describe().show()

+-------+------+
|summary| grade|
+-------+------+
|  count|249054|
|   mean|  null|
| stddev|  null|
|    min|     A|
|    max|     G|
+-------+------+



In [71]:
catcols = [\
           'emp_length',\
           'home_ownership',\
           'verification_status',\
           #'purpose',\
           'grade',\
           'addr_state'\
          ]

num_cols = [\
            #'dti',\
            'loan_amnt',\
            'int_rate',\
            'installment',\
            'annual_inc',\
            #'revol_bal',\
            #'delinq_2yrs',\
            'pub_rec',\
            #'revol_util'\
            'total_acc'\
           ]

discols = [\
           #'pub_rec',\
           'dti',\
           #'installment'\
          ]



labelCol = 'loan_status'

splits = [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, float("inf")]

In [72]:
from itertools import combinations, chain 
list(combinations(num_cols, 2))

[('loan_amnt', 'int_rate'),
 ('loan_amnt', 'installment'),
 ('loan_amnt', 'annual_inc'),
 ('loan_amnt', 'pub_rec'),
 ('loan_amnt', 'total_acc'),
 ('int_rate', 'installment'),
 ('int_rate', 'annual_inc'),
 ('int_rate', 'pub_rec'),
 ('int_rate', 'total_acc'),
 ('installment', 'annual_inc'),
 ('installment', 'pub_rec'),
 ('installment', 'total_acc'),
 ('annual_inc', 'pub_rec'),
 ('annual_inc', 'total_acc'),
 ('pub_rec', 'total_acc')]

In [73]:
for k in range(len(num_cols)):
    for l in (list(combinations(num_cols, k+1))):
        print(list(l))
            

['loan_amnt']
['int_rate']
['installment']
['annual_inc']
['pub_rec']
['total_acc']
['loan_amnt', 'int_rate']
['loan_amnt', 'installment']
['loan_amnt', 'annual_inc']
['loan_amnt', 'pub_rec']
['loan_amnt', 'total_acc']
['int_rate', 'installment']
['int_rate', 'annual_inc']
['int_rate', 'pub_rec']
['int_rate', 'total_acc']
['installment', 'annual_inc']
['installment', 'pub_rec']
['installment', 'total_acc']
['annual_inc', 'pub_rec']
['annual_inc', 'total_acc']
['pub_rec', 'total_acc']
['loan_amnt', 'int_rate', 'installment']
['loan_amnt', 'int_rate', 'annual_inc']
['loan_amnt', 'int_rate', 'pub_rec']
['loan_amnt', 'int_rate', 'total_acc']
['loan_amnt', 'installment', 'annual_inc']
['loan_amnt', 'installment', 'pub_rec']
['loan_amnt', 'installment', 'total_acc']
['loan_amnt', 'annual_inc', 'pub_rec']
['loan_amnt', 'annual_inc', 'total_acc']
['loan_amnt', 'pub_rec', 'total_acc']
['int_rate', 'installment', 'annual_inc']
['int_rate', 'installment', 'pub_rec']
['int_rate', 'installment', 't

In [74]:
spark_labeling = udf(lambda x: "Charged Off" if x == 1.0 else "Fully Paid")

In [75]:
result_list = []

In [106]:
for i in range(len(catcols)):
    for j in (list(combinations(catcols, i+1))):
        for k in range(len(num_cols)):
            for l in (list(combinations(num_cols, k+1))):
                l_list = list(l)
                if ('int_rate' in l_list) & ('installment' in l_list) \
                & ('home_ownership' in j) & ('grade' in j):
                    print (l_list); print(j)

['int_rate', 'installment']
('home_ownership', 'grade')
['loan_amnt', 'int_rate', 'installment']
('home_ownership', 'grade')
['int_rate', 'installment', 'annual_inc']
('home_ownership', 'grade')
['int_rate', 'installment', 'pub_rec']
('home_ownership', 'grade')
['int_rate', 'installment', 'total_acc']
('home_ownership', 'grade')
['loan_amnt', 'int_rate', 'installment', 'annual_inc']
('home_ownership', 'grade')
['loan_amnt', 'int_rate', 'installment', 'pub_rec']
('home_ownership', 'grade')
['loan_amnt', 'int_rate', 'installment', 'total_acc']
('home_ownership', 'grade')
['int_rate', 'installment', 'annual_inc', 'pub_rec']
('home_ownership', 'grade')
['int_rate', 'installment', 'annual_inc', 'total_acc']
('home_ownership', 'grade')
['int_rate', 'installment', 'pub_rec', 'total_acc']
('home_ownership', 'grade')
['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'pub_rec']
('home_ownership', 'grade')
['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'total_acc']
('home_ownership',

In [ ]:
for i in range(len(catcols)):
    for j in (list(combinations(catcols, i+1))):
        for k in range(len(num_cols)):
            for l in (list(combinations(num_cols, k+1))):
                l_list = list(l)
                if ('int_rate' in l_list) & ('installment' in l_list) \
                & ('home_ownership' in j) & ('grade' in j):
                    print("label => ",labelCol)
                    ml_model = get_model(training,j,l_list,discols, splits, labelCol)
                    lap2 = ml_model.transform(test)\
                    .withColumn('prediction_label',spark_labeling(col('prediction')))\
                    .toPandas()
                    m2 = eval_metrics(lap2)
                    result_text = "รอบแรกใช้ categorical {0} และ numerical {1} ได้ Model ที่มี {2}".format(j,l_list,m2)
                    print(result_text)
                    result_list.append(result_text)
                

label =>  loan_status


รอบแรกใช้ categorical ('home_ownership', 'grade') และ numerical ['int_rate', 'installment'] ได้ Model ที่มี {'PPV': 0.6187310098302056, 'Sensitivity': 0.5927333744264092, 'NPV': 0.6494325718496684, 'Specificity': 0.6738026790629397, 'Accuracy': 0.6355573505654281}
label =>  loan_status


รอบแรกใช้ categorical ('home_ownership', 'grade') และ numerical ['loan_amnt', 'int_rate', 'installment'] ได้ Model ที่มี {'PPV': 0.6150697240989117, 'Sensitivity': 0.6328676118074105, 'NPV': 0.6634225975575299, 'Specificity': 0.6462780598201725, 'Accuracy': 0.639951534733441}
label =>  loan_status


รอบแรกใช้ categorical ('home_ownership', 'grade') และ numerical ['int_rate', 'installment', 'annual_inc'] ได้ Model ที่มี {'PPV': 0.60787457116291, 'Sensitivity': 0.6492363536744059, 'NPV': 0.666471296929439, 'Specificity': 0.6259710074010643, 'Accuracy': 0.6369466882067851}
label =>  loan_status


รอบแรกใช้ categorical ('home_ownership', 'grade') และ numerical ['int_rate', 'installment', 'pub_rec'] ได้ Model ที่มี {'PPV': 0.6007483285284917, 'Sensitivity': 0.6707759742483391, 'NPV': 0.6718099269474977, 'Specificity': 0.6018716741085082, 'Accuracy': 0.63437802907916}
label =>  loan_status


...

รอบแรกใช้ categorical ('emp_length', 'home_ownership', 'verification_status', 'grade', 'addr_state') และ numerical ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'total_acc'] ได้ Model ที่มี {'PPV': 0.5880742003823697, 'Sensitivity': 0.38973357989178825, 'NPV': 0.5811455566784968, 'Specificity': 0.7561930393296226, 'Accuracy': 0.5833117932148627}
label =>  loan_status


รอบแรกใช้ categorical ('emp_length', 'home_ownership', 'verification_status', 'grade', 'addr_state') และ numerical ['loan_amnt', 'int_rate', 'installment', 'pub_rec', 'total_acc'] ได้ Model ที่มี {'PPV': 0.5990790344910613, 'Sensitivity': 0.681631395109924, 'NPV': 0.6757689893283113, 'Specificity': 0.5926050522967765, 'Accuracy': 0.6346042003231018}
label =>  loan_status


รอบแรกใช้ categorical ('emp_length', 'home_ownership', 'verification_status', 'grade', 'addr_state') และ numerical ['int_rate', 'installment', 'annual_inc', 'pub_rec', 'total_acc'] ได้ Model ที่มี {'PPV': 0.6064076170869789, 'Sensitivity': 0.6455722210807479, 'NPV': 0.6640919122419836, 'Specificity': 0.6257875099394459, 'Accuracy': 0.6351211631663974}
label =>  loan_status


21/11/25 15:49:28 WARN org.apache.spark.scheduler.AsyncEventQueue: Dropped 2931 events from executorManagement since Thu Nov 25 15:39:33 UTC 2021.
21/11/25 15:50:04 WARN org.apache.spark.ExecutorAllocationManager: Should have exactly one resource profile for stage Stage 108335 (Attempt 0), but have Set()
21/11/25 15:50:05 WARN org.apache.spark.ExecutorAllocationManager: Should have exactly one resource profile for stage Stage 108344 (Attempt 0), but have Set()
21/11/25 15:50:05 WARN org.apache.spark.ExecutorAllocationManager: Should have exactly one resource profile for stage Stage 108344 (Attempt 0), but have Set()
21/11/25 15:50:05 WARN org.apache.spark.ExecutorAllocationManager: Should have exactly one resource profile for stage Stage 108344 (Attempt 0), but have Set()
21/11/25 15:50:05 WARN org.apache.spark.ExecutorAllocationManager: Should have exactly one resource profile for stage Stage 108344 (Attempt 0), but have Set()
21/11/25 15:50:05 WARN org.apache.spark.ExecutorAllocation

รอบแรกใช้ categorical ('emp_length', 'home_ownership', 'verification_status', 'grade', 'addr_state') และ numerical ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'pub_rec', 'total_acc'] ได้ Model ที่มี {'PPV': 0.5756665988194586, 'Sensitivity': 0.7748099445243476, 'NPV': 0.7089750398300584, 'Specificity': 0.48993822252125513, 'Accuracy': 0.624329563812601}


In [83]:
sc

<SparkContext master=yarn appName=PySparkShell>